In [ ]:
# combine two csv with google sheet, remove duplicates 
# datawrapper to identifies lat and lng that were identified outside of new york city
# Remove duplicate in google sheet
# Remove spams manually (would use gpt to identify specific patten next time)

In [4]:
# import
import pandas as pd
import datetime 
import altair as alt
import re

In [6]:
df_all = pd.read_csv('data/mconnections_july_3_final.csv')
df_all.sample(5)

,url,date,title,lat,lng,post
98,https://newyork.craigslist.org/que/mis/d/long-...,2024-06-14 9:06,\nNeed my Latino Bussy Opened Slowly (LIC),40.748284,-73.942280,I host only Looking for a top bunk man to come...
131,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-07 15:24,\nto the cute guy at Sophie concert / Webster ...,40.731323,-73.988613,You were the cute guy from Connecticut at the ...
170,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-07 8:24,\nGuy Qtrain 86/2nd - lost contact,40.791584,-73.940392,We met on 86 2nd Q train late I was the olderg...
162,https://newyork.craigslist.org/brk/mis/d/brook...,2024-06-03 6:23,\nartsy swf spotted tall/slim long haired bear...,40.709500,-73.956300,You are just my type lol
24,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-26 17:57,\nWoman meeting her parents for 50th anniversa...,40.736769,-73.990431,I sat next to you at the uptown Q at Union Squ...


In [8]:
# filter with only June data
df = df_all[df_all['date'] >= '2024-06-01']

# # Convert date object to datetime
df['date'] = pd.to_datetime(df['date'])

# separate date and time in datetime object
df['Date'] = df['date'].dt.date
df['Time'] = df['date'].dt.time

In [10]:
# # convert both Date and Time into datetime object
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S')
df['Weekday'] = df['Date'].dt.weekday
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   url      182 non-null    object        
 1   date     182 non-null    datetime64[ns]
 2   title    182 non-null    object        
 3   lat      182 non-null    float64       
 4   lng      182 non-null    float64       
 5   post     179 non-null    object        
 6   Date     182 non-null    datetime64[ns]
 7   Time     182 non-null    datetime64[ns]
 8   Weekday  182 non-null    int32         
dtypes: datetime64[ns](3), float64(2), int32(1), object(3)
memory usage: 12.2+ KB


In [11]:
# create a new colum for clean title
df['title_clean'] = df['title'].str.replace("\n",'').str.strip().str.lower()
df.sample(5)

,url,date,title,lat,lng,post,Date,Time,Weekday,title_clean
70,https://newyork.craigslist.org/brk/mis/d/new-y...,2024-06-19 09:33:00,\nChar (Brooklyn),40.726200,-73.979600,We met at Max fish we dated for a few months w...,2024-06-19,1900-01-01 09:33:00,2,char (brooklyn)
151,https://newyork.craigslist.org/que/mis/d/brook...,2024-06-04 20:06:00,\nlocked eyes twice w/ a friendly smile (Brook...,40.704400,-73.921500,you know who you are you were wearing a blue a...,2024-06-04,1900-01-01 20:06:00,1,locked eyes twice w/ a friendly smile (brooklyn)
99,https://newyork.craigslist.org/brk/mis/d/brook...,2024-06-14 00:30:00,\nwe met on harman street (bushwick),40.700600,-73.918000,3,2024-06-14,1900-01-01 00:30:00,4,we met on harman street (bushwick)
129,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-08 01:45:00,\nEnjoyable Ride and Chat (Harlem / Morningside),40.858300,-73.929600,We shared an Uber going uptown You are an Ivy ...,2024-06-08,1900-01-01 01:45:00,5,enjoyable ride and chat (harlem / morningside)
80,https://newyork.craigslist.org/que/mis/d/astor...,2024-06-17 18:03:00,\nCollege Asian Guy,40.761977,-73.915479,ou were straight curious and used to stop by m...,2024-06-17,1900-01-01 18:03:00,0,college asian guy


#### I thought I could use the location that was atatched to the title... and went down a rabbit hole of regex. (bad idea)

In [ ]:
# mistake 
# nh = []
# for i in df_june['title_clean']: 
#     onewordRegex = re.compile(r"\(([A-Za-z]+([A-Za-z]+)+)\)") 
#     twowordRegex = re.compile(r"\(([A-Za-z]+( [A-Za-z]+)+)\)") 
#     twowordsRegex = re.compile(r"\(([A-Za-z]+(/ [A-Za-z]+)+)\)") # (e.g. (Nolita / West village), (Nolita/West Village), (bed-stuy))
#     print(i)
#     nh_one = onewordRegex.search(i) 
#     if nh_one is None: 
#         nh_two = twowordRegex.search(i)
#         if nh_two is None: 
#             nh.append('no neighborhood')
#             print('--------', i)
#             continue 
#         print('nh_two append',nh_two)
#         nh.append(nh_two.group().replace("(",'').replace(")",''))
#         continue
#     nh.append(nh_one.group().replace("(",'').replace(")",''))
# print(nh)

In [26]:
# need to use geocoder to get borough data from lat and lng
# !pip3 install opencage
from opencage.geocoder import OpenCageGeocode
from pprint import pprint
# ## figuring out geocoder
# key = ['API KEY HERE']
# geocoder = OpenCageGeocode(key)
# results = geocoder.reverse_geocode(40.6974, -73.9421)
# results[0]['components']

In [14]:
borough_list = []
for i in range(len(df)):
    borough = ''
    lat = df.iloc[i]['lat']
    lng = df.iloc[i]['lng']
    print(lat, lng)
    result = geocoder.reverse_geocode(lat, lng)
    try:
        borough = result[0]['components']['borough']
        borough_list.append(borough)
    except KeyError:
        borough = result[0]['components']['suburb'] #because some areas in queens is not in borough...
        borough_list.append(borough)
    # print(borough)

print(borough_list)

40.7262 -73.9796
40.80932 -73.943975
40.6451 -73.945
40.6505 -73.9349
40.763355 -73.932667
40.7012 -73.9436
40.712484 -74.012221
40.786201 -73.973866
40.7192 -73.9563
40.7255 -73.9983
40.7029 -73.8111
40.723 -73.9321
40.791584 -73.940392
40.770104 -73.969274
40.6907 -73.9274
40.7517 -73.9707
40.7208 -73.7433
40.726925 -73.982898
40.8409 -73.8794
40.8194 -73.8606
40.7717 -73.9717
40.764342 -73.986869
40.6816 -73.9798
40.888201 -73.907597
40.736769 -73.990431
40.717835 -73.957725
40.6784 -73.9211
40.732486 -73.932495
40.7036 -73.8961
40.7351 -73.8776
40.7513 -73.883
40.73373 -73.98468
40.6111 -73.768889
40.656544 -74.012809
40.7547 -73.9925
40.6006 -73.758
40.6669 -73.9828
40.6907 -73.9274
40.7095 -73.9563
40.724269 -73.984837
40.678513 -73.902283
40.6907 -73.9274
40.5839 -73.9536
40.6039 -74.1472
40.8246 -73.9507
40.5518 -74.1526
40.6669 -73.9828
40.820805 -73.868766
40.7627 -73.7745
40.8496 -73.9356
40.756646 -73.989036
40.8183 -73.9426
40.816552 -73.947773
40.753007 -73.977127
40.7075

In [15]:
# put all the borough into the borough column
df['borough'] = borough_list

In [16]:
# check the borough list
df['borough'].value_counts()

borough
Manhattan        89
Brooklyn         51
Queens County    21
The Bronx        10
Staten Island     8
Queens            2
Kings County      1
Name: count, dtype: int64

In [17]:
df["borough"] = df["borough"].replace(["Kings County"], "Brooklyn")
df["borough"] = df["borough"].replace(["Queens County"], "Queens")
df['borough'].value_counts()

borough
Manhattan        89
Brooklyn         52
Queens           23
The Bronx        10
Staten Island     8
Name: count, dtype: int64

In [25]:
base = alt.Chart(pd.DataFrame(df['borough'].value_counts().reset_index())).encode(
    x='count',
    y=alt.Y('borough').sort('-x'),
    # text='borough'
)
base.mark_bar() + base.mark_text(align='left', dx=2)

alt.LayerChart(...)

#### tokenize post data

In [28]:
import spacy
import nltk
from nltk.corpus import stopwords
nlp = spacy.load("en_core_web_sm")
stops = stopwords.words("english")

In [29]:
def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        print(word.lemma_.strip())
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return lemmatized

In [31]:
## test the normalize function
#text_norm = normalize(df.iloc[2]['post'], lowercase=True, remove_stopwords=True)
## test out stemming
#list_of_strings  = [i for i in text_norm]

In [32]:
# Define a function to perform tokenization
def tokenize_text(text):
    doc = nlp(str(text))
    return [token for token in doc]

# Apply the tokenization function to the 'post' column and store the results in a new column
df['tokenized_post'] = df['post'].apply(tokenize_text)
df.head(10)

,url,date,title,lat,lng,post,Date,Time,Weekday,title_clean,borough,tokenized_post
0,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-30 23:47:00,\nPunjabi deli (Lower East Side),40.726200,-73.979600,I was standing in line waiting for my curry we...,2024-06-30,1900-01-01 23:47:00,6,punjabi deli (lower east side),Manhattan,"[I, was, standing, in, line, waiting, for, my,..."
1,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-30 15:37:00,\nMelanated beauty working at Sky Jewelry Stor...,40.809320,-73.943975,I came in today Sunday with a black CROWN hat ...,2024-06-30,1900-01-01 15:37:00,6,melanated beauty working at sky jewelry store ...,Manhattan,"[I, came, in, today, Sunday, with, a, black, C..."
2,https://newyork.craigslist.org/brk/mis/d/brook...,2024-06-30 15:23:00,\nRave Bae “Cat” at John Summit Navy Yard,40.645100,-73.945000,I was the guy with the curly hair wearing all ...,2024-06-30,1900-01-01 15:23:00,6,rave bae “cat” at john summit navy yard,Brooklyn,"[I, was, the, guy, with, the, curly, hair, wea..."
3,https://newyork.craigslist.org/brk/mis/d/brook...,2024-06-30 11:34:00,\nLooking for Amir (Brooklyn),40.650500,-73.934900,Hi,2024-06-30,1900-01-01 11:34:00,6,looking for amir (brooklyn),Brooklyn,[Hi]
4,https://newyork.craigslist.org/brk/mis/d/astor...,2024-06-30 10:42:00,\nMissed connection with JB on CazChat (Brookl...,40.763355,-73.932667,Not sure what just happened but the site clinc...,2024-06-30,1900-01-01 10:42:00,6,missed connection with jb on cazchat (brooklyn),Queens,"[Not, sure, what, just, happened, but, the, si..."
5,https://newyork.craigslist.org/brk/mis/d/brook...,2024-06-30 01:09:00,\nGirl on L train with black and orange shirt.,40.701200,-73.943600,You sat across from me You had on a black shir...,2024-06-30,1900-01-01 01:09:00,6,girl on l train with black and orange shirt.,Brooklyn,"[You, sat, across, from, me, You, had, on, a, ..."
6,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-30 00:24:00,\nPink Nails at the Jellicle Ball 6/29 (Downto...,40.712484,-74.012221,We eyed one another throughout and both had th...,2024-06-30,1900-01-01 00:24:00,6,pink nails at the jellicle ball 6/29 (downtown),Manhattan,"[We, eyed, one, another, throughout, and, both..."
7,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-29 22:52:00,\nMiddle age cute man (Upper West Side),40.786201,-73.973866,You are the middle age man I stared outside of...,2024-06-29,1900-01-01 22:52:00,5,middle age cute man (upper west side),Manhattan,"[You, are, the, middle, age, man, I, stared, o..."
8,https://newyork.craigslist.org/brk/mis/d/brook...,2024-06-29 20:05:00,"\nHandsome, 7.30, 2x (Berry, Bedford) (Brooklyn)",40.719200,-73.956300,Grungy shirt noticed your face more than your ...,2024-06-29,1900-01-01 20:05:00,5,"handsome, 7.30, 2x (berry, bedford) (brooklyn)",Brooklyn,"[Grungy, shirt, noticed, your, face, more, tha..."
9,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-29 18:47:00,\nCute guy on C train uptown 5:15 today (Downt...,40.725500,-73.998300,You were in a black t shirt black shorts white...,2024-06-29,1900-01-01 18:47:00,5,cute guy on c train uptown 5:15 today (downtown),Manhattan,"[You, were, in, a, black, t, shirt, black, sho..."


In [33]:
def get_verb(word):
    verb = []
    for token in word:
        # print(type(token))
        if token.pos_ == "VERB":
            verb.append(token.lemma_)
        else: 
            continue
    return verb

df['verb'] = df['tokenized_post'].apply(get_verb)
df.sample(5)

,url,date,title,lat,lng,post,Date,Time,Weekday,title_clean,borough,tokenized_post,verb
171,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-18 14:26:00,\nWhere are you (Upper East Side),40.776300,-73.952900,Normal guy looking for a buddy who s into expl...,2024-06-18,1900-01-01 14:26:00,1,where are you (upper east side),Manhattan,"[Normal, guy, looking, for, a, buddy, who, s, ...","[look, s, explore]"
36,https://newyork.craigslist.org/brk/mis/d/brook...,2024-06-24 01:02:00,"\nJoe's Pizza Park Slope, you told me about DJ...",40.666900,-73.982800,I went to Public Records later on Friday nite ...,2024-06-24,1900-01-01 01:02:00,0,"joe's pizza park slope, you told me about dj a...",Brooklyn,"[I, went, to, Public, Records, later, on, Frid...","[go, hope, run, speak, invite, sit, say]"
96,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-14 20:47:00,\nGrey Poodle Owner on Carl Schurz Park (Upper...,40.774527,-73.944297,Three weeks ago I was dog sitting my friend s ...,2024-06-14,1900-01-01 20:47:00,4,grey poodle owner on carl schurz park (upper e...,Manhattan,"[Three, weeks, ago, I, was, dog, sitting, my, ...","[sit, walk, run, come, comment, leave, mention..."
109,https://newyork.craigslist.org/brk/mis/d/brook...,2024-06-11 21:42:00,\nLooking at an apartment in cobble hill (Broo...,40.692739,-73.990002,We arrived to look at an apartment at the same...,2024-06-11,1900-01-01 21:42:00,1,looking at an apartment in cobble hill (brooklyn),Brooklyn,"[We, arrived, to, look, at, an, apartment, at,...","[arrive, look, wear, want, hang, talk, have, g..."
64,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-19 19:23:00,\nMaya at Jake's Dilemma (Upper West Side),40.786400,-73.976400,You were with your white friend but ended up t...,2024-06-19,1900-01-01 19:23:00,2,maya at jake's dilemma (upper west side),Manhattan,"[You, were, with, your, white, friend, but, en...","[end, talk, s, get, talk, go, give]"


In [34]:
def get_noun(word):
    noun = []
    for token in word:
        # print(type(token))
        if token.pos_ == "NOUN":
            noun.append(token.lemma_)
        else: 
            continue
    return noun

df['noun'] = df['tokenized_post'].apply(get_noun)
df.sample(5)

,url,date,title,lat,lng,post,Date,Time,Weekday,title_clean,borough,tokenized_post,verb,noun
147,https://newyork.craigslist.org/brx/mis/d/bronx...,2024-06-04 23:30:00,\nTo the MTA Bx9 Bus Driver Tues 6/04 around 8...,40.8798,-73.9067,This seems like a long shot but putting this o...,2024-06-04,1900-01-01 23:30:00,1,to the mta bx9 bus driver tues 6/04 around 8:1...,The Bronx,"[This, seems, like, a, long, shot, but, puttin...","[seem, put, know, have, work, pass]","[shot, case, access, schedule, driver, route, ..."
74,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-18 13:37:00,\nKey food (East Village),40.7262,-73.9796,On 4th street Key food you were the guy at del...,2024-06-18,1900-01-01 13:37:00,1,key food (east village),Manhattan,"[On, 4th, street, Key, food, you, were, the, g...",[],"[street, food, guy, counter, night, pm]"
164,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-01 00:26:00,\nWe Shared a Bus Ride Home (Upper East Side),40.7685,-73.9588,You wore black and white or cream,2024-06-01,1900-01-01 00:26:00,5,we shared a bus ride home (upper east side),Manhattan,"[You, wore, black, and, white, or, cream]","[wear, cream]",[]
176,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-26 04:22:00,\nYou were coming out of the true value hardwa...,40.7685,-73.9588,You were coming out of the true value hardware...,2024-06-26,1900-01-01 04:22:00,2,you were coming out of the true value hardware...,Manhattan,"[You, were, coming, out, of, the, true, value,...","[come, wear, comment, say, look, say, like, th...","[value, hardware, store, hat, sneaker, man, sh..."
11,https://newyork.craigslist.org/que/mis/d/brook...,2024-06-29 15:50:00,\nTeddy at Horsegiirl (Brooklyn),40.7230,-73.9321,We talked about Giorgio moroder and Wilco unde...,2024-06-29,1900-01-01 15:50:00,5,teddy at horsegiirl (brooklyn),Brooklyn,"[We, talked, about, Giorgio, moroder, and, Wil...","[talk, tell, say, look, hang]","[moroder, bridge, land, night, celebrity]"


In [35]:
def get_adj(word):
    adj = []
    for token in word:
        # print(type(token))
        if token.pos_ == "ADJ":
            adj.append(token.lemma_)
        else: 
            continue
    return adj

df['adj'] = df['tokenized_post'].apply(get_adj)
df.sample(5)

,url,date,title,lat,lng,post,Date,Time,Weekday,title_clean,borough,tokenized_post,verb,noun,adj
106,https://newyork.craigslist.org/brx/mis/d/bronx...,2024-06-12 09:58:00,\nBronx 2 train 233 to Pelham (Bronx),40.90000,-73.862200,You got on the train this morning I believe at...,2024-06-12,1900-01-01 09:58:00,2,bronx 2 train 233 to pelham (bronx),The Bronx,"[You, got, on, the, train, this, morning, I, b...","[get, believe, wear, have, get, be, hit, love,...","[train, morning, glass, tee, leg, seat, tee, i...","[cute, beard, green, left, gray, mutual]"
132,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-07 14:52:00,"\nSaw You on the L, then You Said Hi to me Hou...",40.72550,-73.998300,I thought you were someone I talk to on Fetlif...,2024-06-07,1900-01-01 14:52:00,4,"saw you on the l, then you said hi to me hours...",Manhattan,"[I, thought, you, were, someone, I, talk, to, ...","[think, talk, seem, stare, stare, pass, say]","[work, people]",[]
71,https://newyork.craigslist.org/stn/mis/d/state...,2024-06-19 09:11:00,\nVan Brezen Park (Varrazano Bridge),40.59730,-74.076800,We spoke in the parking lot and I felt a conne...,2024-06-19,1900-01-01 09:11:00,2,van brezen park (varrazano bridge),Staten Island,"[We, spoke, in, the, parking, lot, and, I, fel...","[speak, feel]","[parking, lot, connection]",[]
92,https://newyork.craigslist.org/brx/mis/d/bronx...,2024-06-15 12:27:00,\nAt GameStop this morning (parkchester),40.84340,-73.860200,I was so focused on getting what I was getting...,2024-06-15,1900-01-01 12:27:00,5,at gamestop this morning (parkchester),The Bronx,"[I, was, so, focused, on, getting, what, I, wa...","[get, get, forget, ask, damn, want, want, link...","[gamertag, freaking, head, game]","[focused, gorgeous, low]"
136,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-06 19:07:00,\nSunday consult for a patient with spots,40.74124,-73.975164,I asked you about all the running around you d...,2024-06-06,1900-01-01 19:07:00,3,sunday consult for a patient with spots,Manhattan,"[I, asked, you, about, all, the, running, arou...","[ask, d, do, ask, watch, impair, think, seem, ...","[running, day, stuff, coffee, compliment, eye,...","[cute, smart, bold, sure, sure, light]"


In [38]:
# see if its possible to translate title and post into emoji
# !pip install emot
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 2.8 MB/s eta 0:00:00a 0:00:01


In [39]:
import emoji

In [49]:
# text to emoji example
# text = 'I love Python :red_heart:, it is brilliant :thumbs_up: '
# emoji.emojize(text)

you = ':train:'
emoji.emojize(you)

'🚆'

In [69]:
def word_to_emoji(word):
    # Try to find an emoji for the word
    emoji_str = f":{word.lower()}:"
    e = emoji.emojize(emoji_str)
    if e != emoji_str:
        return e
    else: 
        pass # why cant i use continue here?

# ## test the word_to_emoji function
# words = ["You", "got", "on", "the", "train", "this", "morning"]

# # Convert words to emojis where possible
# emojis = [word_to_emoji(word) for word in words]

# # Print the resulting list
# print(emojis)

[None, None, None, None, '🚆', None, None]


In [77]:
## test the word_to_emoji function on first row's text
emojis = [word_to_emoji(word) for word in df.iloc[0]['noun']]
print(emojis)

[None, None, None, '👁️', None, None]


In [106]:
## test the word_to_emoji function on dataframe verb
for index, row in df.iterrows():
    p = row['title_clean']
    text = p.split()
    print(text)
    emojis = [word_to_emoji(word) for word in text]
    print(emojis)

['punjabi', 'deli', '(lower', 'east', 'side)']
[None, None, None, None, None]
['melanated', 'beauty', 'working', 'at', 'sky', 'jewelry', 'store', '(harlem', '/', 'morningside)']
[None, None, None, None, None, None, None, None, None, None]
['rave', 'bae', '“cat”', 'at', 'john', 'summit', 'navy', 'yard']
[None, None, None, None, None, None, None, None]
['looking', 'for', 'amir', '(brooklyn)']
[None, None, None, None]
['missed', 'connection', 'with', 'jb', 'on', 'cazchat', '(brooklyn)']
[None, None, None, None, None, None, None]
['girl', 'on', 'l', 'train', 'with', 'black', 'and', 'orange', 'shirt.']
['👧', None, None, '🚆', None, None, None, None, None]
['pink', 'nails', 'at', 'the', 'jellicle', 'ball', '6/29', '(downtown)']
[None, None, None, None, None, None, None, None]
['middle', 'age', 'cute', 'man', '(upper', 'west', 'side)']
[None, None, None, '👨', None, None, None]
['handsome,', '7.30,', '2x', '(berry,', 'bedford)', '(brooklyn)']
[None, None, None, None, None, None]
['cute', 'guy',

In [105]:
# # test out other text to emoji libraries
# !pip uninstall emojipy 
## trying to solve the None problem...decided to look at the most common word and manually create a dictionary

# most used verb
# get      53
# see      44
# look     39
# have     31
# sit      29
# wear     28
# say      25
# think    21
# m        21
# make     20
# go       19
# know     18
# hope     17
# like     16
# want     16
# smile    15
# talk     15
# meet     15
# s        15
# ask      15

# most used adj
# black        28
# other        15
# white        14
# beautiful    14
# cute         10
# late          9
# blue          9
# same          9
# few           8
# old           8
# last          8
# sure          7
# nice          7
# long          7
# tall          6
# red           5
# dark          5
# good          5
# handsome      4
# stunning      4

# most used noun
# eye        35
# guy        30
# train      29
# time       22
# friend     18
# contact    15
# hair       15
# t          13
# woman      13
# shirt      13
# girl       12
# dress      12
# number     11
# night      10
# pm         10
# smile       9
# male        8
# short       8
# day         8
# hat         8

Found existing installation: emojipy 3.0.5
Uninstalling emojipy-3.0.5:
  Would remove:
    /Users/jy/miniconda3/lib/python3.12/site-packages/emojipy-3.0.5.dist-info/*
    /Users/jy/miniconda3/lib/python3.12/site-packages/emojipy/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [175]:
## putting gpt to the work
word_to_emoji_dict = {
    # "eye": ":eye:",
    "guy": ":man:",
    "train": ":train2:",
    "time": ":watch:",
    "friend": ":busts_in_silhouette:",
    "contact": ":mobile_phone:",
    "hair": ":red_hair:",
    # "woman": ":woman:",
    # "shirt": ":shirt:",
    # "girl": ":girl:",
    # "dress": ":dress:",
    "number": ":input_numbers:",
    "night": ":night_with_stars:",
    "smile": ":slightly_smiling_face:",
    "male": ":man:",
    "short": ":shorts:",
    "day": ":sun_with_face:",
    "hat": ":top_hat:",
    "run": ":person_running:",
    "black": ":black_circle:",
    # "other": ":busts_in_silhouette:",
    "white": ":white_circle:",
    "beautiful": ":sparkles:",
    "cute": ":smiling_face_with_heart-eyes:",
    "late": ":alarm_clock:",
    "blue": ":blue_circle:",
    "same": ":repeat_button:",
    # "few": ":three:",
    # "old": ":older_adult:",
    "last": ":last_quarter_moon:",
    "sure": ":thumbs_up:",
    # "nice": ":blush:",
    "long": ":straight_ruler:",
    "tall": ":giraffe:",
    "red": ":red_circle:",
    "dark": ":new_moon:",
    "good": ":OK_hand:",
    "handsome": ":man:",
    "stunning": ":star-struck:",
    "see": ":eyes:",
    "look": ":eyes:",
    "have": ":check_mark:",
    "sit": ":seat:",
    "wear": ":t-shirt:",
    "say": ":speech_balloon:",
    "think": ":thought_balloon:",
    "make": ":hammer_and_wrench:",
    "go": ":right_arrow:",
    "know": ":brain:",
    "hope": ":crossed_fingers:",
    "like": ":thumbs_up:",
    # "want": ":point_right:",
    "talk": ":speaking_head:",
    "conversation": ":speaking_head:",
    "meet": ":handshake:",
    "ask": ":red_question_mark:",
    "pm": ":sunset:", 
    "morning": ":sunrise:",
}

In [178]:
## try to redefine word to emojis
def word_to_emojis(word):
    # Try to find an emoji for the word
    s = ""
    emoji_str = f":{word.lower()}:"
    # print('----', emoji_str)
    e = emoji.emojize(emoji_str)
    # print(e)
    if e != emoji_str:
        s = e
        return s
    elif e == emoji_str: 
        # # look into the word_to_emoji_dict
        try: 
            e_dict = emoji.emojize(word_to_emoji_dict[word.lower()])
            # print(e_dict)
            # if e_dict == emoji_str: 
            #     print()
            s = e_dict
            return s
        # # except when the word also 
        except KeyError: 
            pass
        else: 
            pass # why cant i use continue here?
        

In [179]:
emojis = [word_to_emojis(word) for word in df.iloc[0]['noun']]
print(emojis)

[None, None, '🗣️', '👁️', None, None]


In [191]:
df['emojis_adj'] = df['adj']
df['emojis_noun'] = df['noun']
df['emojis_verb'] = df['verb']
# # df.sample(5)
for index, row in df.iterrows():
    # p = row['verb']
    # text = p.split()
    # print(text)
    emojis_adj = [word_to_emojis(word) for word in row['adj']]
    emojis_verb = [word_to_emojis(word) for word in row['verb']]
    emojis_noun = [word_to_emojis(word) for word in row['noun']]
    # print(emojis)
    df.at[index,'emojis_adj'] = emojis_adj
    df.at[index,'emojis_verb'] = emojis_verb
    df.at[index,'emojis_noun'] = emojis_noun

df.sample(5)
# # df['emojis'] = emojis

# df['emoji'] = df['adj'].apply(lambda x: word_to_emojis(word) for word in x)

,url,date,title,lat,lng,post,Date,Time,Weekday,title_clean,borough,tokenized_post,verb,noun,adj,title_emoji,emojis_adj,emojis_noun,emojis_verb
179,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-26 20:24:00,\nFor the beautiful Latina the needed assistance,40.759700,-73.991800,I wanted to help you out with your situation H...,2024-06-26,1900-01-01 20:24:00,2,for the beautiful latina the needed assistance,Manhattan,"[I, wanted, to, help, you, out, with, your, si...","[want, help, hit, let, discuss]",[situation],[],None,[],[None],"[None, None, None, None, None]"
85,https://newyork.craigslist.org/brx/mis/d/bronx...,2024-06-17 03:15:00,\nDowntown 4 Train (Bronx),40.858368,-73.903908,You are a very handsome guy with nice arms You...,2024-06-17,1900-01-01 03:15:00,0,downtown 4 train (bronx),The Bronx,"[You, are, a, very, handsome, guy, with, nice,...","[take, get, m, sit, sound, hit, let, chat]","[guy, arm, downtown, train, morning, 50am, 86t...","[handsome, nice, familiar]",None,"[👨, None, None]","[👨, None, None, 🚆, 🌅, None, None, 👨]","[None, None, None, 💺, None, None, None, None]"
154,https://newyork.craigslist.org/mnh/mis/d/new-y...,2024-06-04 13:43:00,\nFriendly Veteran Uptown A/C platform 14th st...,40.737299,-73.997040,Saturday afternoon My neck hurt and we started...,2024-06-04,1900-01-01 13:43:00,1,friendly veteran uptown a/c platform 14th stre...,Manhattan,"[Saturday, afternoon, My, neck, hurt, and, we,...","[start, chat, love, continue]","[afternoon, neck, hurt, conversation]",[],None,[],"[None, None, None, 🗣️]","[None, None, None, None]"
30,https://newyork.craigslist.org/que/mis/d/jacks...,2024-06-24 21:28:00,\nTattooed arm (Elmhurst),40.751300,-73.883000,Hey I asked you for a pic of your tattoo You a...,2024-06-24,1900-01-01 21:28:00,0,tattooed arm (elmhurst),Queens,"[Hey, I, asked, you, for, a, pic, of, your, ta...","[ask, ask, finish, see, write, tell]","[pic, tattoo, time]",[cute],None,[😍],"[None, None, ⌚]","[❓, ❓, None, 👀, None, None]"
155,https://newyork.craigslist.org/brk/mis/d/brook...,2024-06-04 09:34:00,\nGluten Free Girl? (Brooklyn),40.676595,-73.902283,Met you once Felt like there was a lot more to...,2024-06-04,1900-01-01 09:34:00,1,gluten free girl? (brooklyn),Brooklyn,"[Met, you, once, Felt, like, there, was, a, lo...","[feel, be, explore, regard, bat, pray, see, re...","[lot, lack, knowledge, figure, eye, year, maki...","[more, certain, political, embarrassing, big, ...",None,"[None, None, None, None, None, 👍, None]","[None, None, None, None, 👁️, None, None, None,...","[None, None, None, None, 🦇, None, 👀, None, Non..."


In [193]:
# combine all three columns
df['emojis'] = df['emojis_adj'] + df['emojis_verb'] + df['emojis_noun']
df.sample(5)
# remove None in all the list
# Function to remove None values from a list
def remove_none(lst):
    return [x for x in lst if x is not None]

# Apply the function to each cell in the 'emojis' column and create a new column
df['cleaned_emojis'] = df['emojis'].apply(remove_none)

df.sample(5)
# save it to a csv

,url,date,title,lat,lng,post,Date,Time,Weekday,title_clean,...,tokenized_post,verb,noun,adj,title_emoji,emojis_adj,emojis_noun,emojis_verb,emojis,cleaned_emojis
45,https://newyork.craigslist.org/stn/mis/d/state...,2024-06-23 01:53:00,\nus 2 guys we were gonna meet you work in gr...,40.551800,-74.152600,i lost your contact id like to meet up im free...,2024-06-23,1900-01-01 01:53:00,6,us 2 guys we were gonna meet you work in grea...,...,"[i, lost, your, contact, i, d, like, to, meet,...","[lose, like, meet, m]","[contact, night, contact]","[free, late, usual]",None,"[None, ⏰, None]","[📱, 🌃, 📱]","[None, 👍, 🤝, None]","[None, ⏰, None, None, 👍, 🤝, None, 📱, 🌃, 📱]","[⏰, 👍, 🤝, 📱, 🌃, 📱]"
97,https://newyork.craigslist.org/que/mis/d/long-...,2024-06-14 12:18:00,"\nSoy marica, tengo hambre (Long Island City Q...",40.748129,-73.942280,Hola soy marica Tengo hambre Quiero guevo Solo...,2024-06-14,1900-01-01 12:18:00,4,"soy marica, tengo hambre (long island city que...",...,"[Hola, soy, marica, Tengo, hambre, Quiero, gue...",[s],"[soy, marica, guevo, diversi]","[para, nos]",None,"[None, None]","[None, None, None, None]",[None],"[None, None, None, None, None, None, None]",[]
38,https://newyork.craigslist.org/brk/mis/d/brook...,2024-06-23 20:53:00,\nGirl at the bar at the Scotland Soccer game ...,40.709500,-73.956300,You were sitting at the bar wearing all black ...,2024-06-23,1900-01-01 20:53:00,6,girl at the bar at the scotland soccer game (w...,...,"[You, were, sitting, at, the, bar, wearing, al...","[sit, wear, keep, make, sit]","[bar, eye, contact, minute, friend, booth, mes...","[black, few, front]",None,"[⚫, None, None]","[None, 👁️, 📱, None, 👥, None, None]","[💺, 👕, None, 🛠️, 💺]","[⚫, None, None, 💺, 👕, None, 🛠️, 💺, None, 👁️, 📱...","[⚫, 💺, 👕, 🛠️, 💺, 👁️, 📱, 👥]"
85,https://newyork.craigslist.org/brx/mis/d/bronx...,2024-06-17 03:15:00,\nDowntown 4 Train (Bronx),40.858368,-73.903908,You are a very handsome guy with nice arms You...,2024-06-17,1900-01-01 03:15:00,0,downtown 4 train (bronx),...,"[You, are, a, very, handsome, guy, with, nice,...","[take, get, m, sit, sound, hit, let, chat]","[guy, arm, downtown, train, morning, 50am, 86t...","[handsome, nice, familiar]",None,"[👨, None, None]","[👨, None, None, 🚆, 🌅, None, None, 👨]","[None, None, None, 💺, None, None, None, None]","[👨, None, None, None, None, None, 💺, None, Non...","[👨, 💺, 👨, 🚆, 🌅, 👨]"
141,https://newyork.craigslist.org/brk/mis/d/new-y...,2024-06-05 19:17:00,\nsolomons on the L (somewhere between union s...,40.715167,-73.971806,dear beautiful man on the L train in the white...,2024-06-05,1900-01-01 19:17:00,2,solomons on the l (somewhere between union sq ...,...,"[dear, beautiful, man, on, the, L, train, in, ...","[hope, get, sort]","[man, l, train, solomon, evening, eye, drop]","[dear, beautiful, white]",None,"[None, ✨, ⚪]","[👨, None, 🚆, None, None, 👁️, None]","[🤞, None, None]","[None, ✨, ⚪, 🤞, None, None, 👨, None, 🚆, None, ...","[✨, ⚪, 🤞, 👨, 🚆, 👁️]"


In [214]:
# df.drop(columns=['title_emoji']).to_csv('mc_final_w_emojis.csv', sep='\t', encoding='utf-8')

In [198]:
# getting all the nouns 
all_nouns = [noun for sublist in df['noun'] for noun in sublist]
unique_nouns = list(set(all_nouns))
print(unique_nouns)

['elevator', 'fact', 'briefly', 'kid', 'olive', 'heart', 'bar', 'bargain', 'hat', 'opportunity', 'love', '30ish', 'honesty', 'cut', 'course', 'way', 'hip', '1st', 'explorer', 'reprise', 'pic', 'skin', 'intrest', 'effort', 'wedding', 's', 'intersection', 'adida', 'movie', 'board', 'sweater', 'tee', 'sight', 'dealer', 'reader', 'second', 'draw', 'phone', 'night', 'hey', 'yesterday', 'strip', 'window', 'mate', 'concert', 'belly', 'weekend', '34th', 'minute', 'leg', 'people', 'cigarette', 'favor', 'drink', 'table', 'headphone', 'food', 'glass', 'life', 'week', 'home', 'tomorrow', 'gamertag', 'f', 'schedule', 'laptop', 'contact', 'side', 'basketball', 'craigslist', 'teller', 'handle', 'sleeve', 'dusk', 'easch', 'station', 'foot', 'club', 'improbability', 'parkchester', 'stop', 'cinder', 'training', 'park', 'buddy', 'reason', 'waist', 'lbs', 'suitcase', 'supermarket', 'point', 'dude', 'lane', 'chick', 'traffic', 'air', 'attempt', 'past', 'nerve', 'chaos', 'ear', 'excuse', 'access', 'office',

In [208]:
# manhattan nouns 

from collections import Counter
import matplotlib.pyplot as plt

manhattan = df[df['borough'] == 'Manhattan']
all_nouns_manhattan = [noun for sublist in manhattan['noun'] for noun in sublist]
all_nouns_manhattan_data = Counter(all_nouns_manhattan)

all_nouns_manhattan_data

# # Extracting the data
# nouns = list(all_nouns_manhattan_data.keys())
# counts = list(all_nouns_manhattan_data.values())

# plt.figure(figsize=(10, 6))
# plt.bar(nouns, counts, color='skyblue')
# plt.xlabel('Nouns')
# plt.ylabel('Counts')
# plt.title('Counts of Nouns in Manhattan')
# plt.xticks(rotation=45, ha='right')
# plt.tight_layout()
# plt.show()

Counter({'eye': 20,
         'guy': 20,
         'train': 19,
         'time': 13,
         'shirt': 10,
         'friend': 10,
         'hair': 9,
         'short': 8,
         'number': 8,
         'dress': 8,
         'hat': 7,
         't': 7,
         'contact': 7,
         'girl': 7,
         'today': 6,
         'man': 6,
         'street': 6,
         'line': 5,
         'pm': 5,
         'afternoon': 5,
         'night': 5,
         'woman': 5,
         'day': 5,
         'name': 4,
         'park': 4,
         'bench': 4,
         'card': 4,
         'yesterday': 4,
         'lot': 4,
         'chance': 4,
         'lady': 4,
         'couple': 4,
         'month': 4,
         'coffee': 4,
         'conversation': 3,
         'place': 3,
         'bit': 3,
         'thing': 3,
         'suit': 3,
         'way': 3,
         'bus': 3,
         'male': 3,
         'belly': 3,
         'front': 3,
         'lunch': 3,
         'shoe': 3,
         'pant': 3,
         'd': 3,
    

In [212]:
df_exploded = df.explode('noun')
# Group by 'borough' and get the value counts for 'nouns'
value_counts = df_exploded.groupby('borough')['noun'].value_counts().unstack(fill_value=0)
value_count


##### noun analysis


noun,13th,1st,20,22nd,30,30ish,30pm,34th,35y,40,...,while,window,wolf,woman,work,year,yer,yesterday,yoga,zip
borough,,,,,,,,,,,,,,,,,,,,,
Brooklyn,0,0,1,0,1,1,1,0,0,1,...,0,0,0,7,1,1,0,0,0,0
Manhattan,0,1,0,0,0,0,1,0,0,0,...,2,1,1,5,2,2,1,4,1,1
Queens,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,2,1,0,0,0,0
Staten Island,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
The Bronx,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
